## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
####P rof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Renzo Antonio Zagni Vestrini
*   A01795457@tec.mx


In [5]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os
from google.colab import drive

import warnings
warnings.filterwarnings("ignore")



In [2]:
# Guardamos el set de datos en Google Drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA"
os.chdir(DIR)

Mounted at /content/drive


### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [99]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer', encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++

El argumento "latin-1" en la opcion encoding es necesario para decodificar correctamente los caracteres especiales de lenguajes basados en caracteres latinos como el español los cuales no son soportados por el encoder 'UTF-8' y que pd.read_csv()  utiliza por defecto.

Desde el punto de vista de este ejercicio, el segundo archivo "geoplaces2.csv" contiene el caracter "ó" representado por el byte 0xf3 el cual no es valido en una secuencia UTF-8 por lo que es necesario utilizar el encoder "latin-1" para que el caracter sea interpretado correctamente

++++++++ Termina la sección de agregar texto. +++++++++++

In [22]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [73]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [24]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [31]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1, data2, on="placeID")


# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 25)


,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


## **Ejercicio - 3**

In [93]:
len(df_combinado['name'].unique())

129

In [101]:
print(UtMx_food.to_string())

userID                                                U1001  U1002  U1003  U1004  U1005  U1006  U1007  U1008  U1009  U1010  U1011  U1012  U1013  U1014  U1015  U1016  U1017  U1018  U1019  U1020  U1021  U1022  U1023  U1024  U1025  U1026  U1027  U1028  U1029  U1030  U1031  U1032  U1033  U1034  U1035  U1036  U1037  U1038  U1039  U1040  U1041  U1042  U1043  U1044  U1045  U1046  U1047  U1048  U1049  U1050  U1051  U1052  U1053  U1054  U1055  U1056  U1057  U1058  U1059  U1060  U1061  U1062  U1063  U1064  U1065  U1066  U1067  U1068  U1069  U1070  U1071  U1072  U1073  U1074  U1075  U1076  U1077  U1078  U1079  U1080  U1081  U1082  U1083  U1084  U1085  U1086  U1087  U1088  U1089  U1090  U1091  U1092  U1093  U1094  U1095  U1096  U1097  U1098  U1099  U1100  U1101  U1102  U1103  U1104  U1105  U1106  U1107  U1108  U1109  U1110  U1111  U1112  U1113  U1114  U1115  U1116  U1117  U1118  U1119  U1120  U1121  U1122  U1123  U1124  U1125  U1126  U1127  U1128  U1129  U1130  U1131  U1132  U1133  U1134  U1135  U

In [87]:
UtMx_food

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tacos de barbacoa enfrente del Tec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos de la estacion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tacos los volcanes,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [90]:
len(list(UtMx_food.index))

129

In [84]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(values='food_rating', index='name', columns='userID', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [ ]:
# Apliquemos la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Deberás obtener la factorización SVD con el máximo de
# comonentes y por lo tanto el máximo de valores singulares posible.
# Asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que podemos obtener de la utilidad que generamos en el ejercicio anterior.


# ************* Inlcuye aquí tu código:*****************************


nc_food = None     # número de componentes


# *********** Aquí termina la sección de agregar código *************


# Inicializamos y aplicamos la factorización SVD en relación a la
# evaluación de la comida, para determinar la cantidad de componentes
# que podemos utilizar con un 90% de explicación de la variabilidad
# de dichas componentes:

SVD_food = TruncatedSVD( n_components = nc_food )
SVD_food.fit(UtMx_food)

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

print('Total de valores singulares basados en evaluación de la :', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)


In [ ]:
# Usando la métrica de correlación de Pearson, obtendremos las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades com el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtendremos las correlaciones positivas y de ahí deberás seleccionar
# las diez mejores.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:7]

## **Ejercicio - 5**

In [ ]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente y llamaremos "mejores_sim_food_ordenadas".
#
# NOTA: Puedes incluir a continuación las líneas de código que requieras.

# ************* Inlcuye aquí tu código:*****************************



mejores_sim_food_ordenadas = None



# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), encuentrarás ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [ ]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************


UtMx_service = None


# *********** Aquí termina la sección de agregar código *************




print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

In [ ]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************


None    # Incluye todas las líneas de código o celdas que requieras.


# *********** Aquí termina la sección de agregar código *************

## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

++++++++ Inicia la sección de agregar texto: +++++++++++


++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**